In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [71]:
class El_Farol_env:
    def __init__(self, NUM_AGENTS = 100, M = 5, N = 10, THR = 60, SEED=123):
        np.random.seed(SEED)
        self. num_agents = NUM_AGENTS # Number of agents
        self.M = M # Number of weeks to remember (memory)
        self.N = N # Number of strategies per agent
        self.THR = 60 # Occupancy threshold
        
        # Initialize random decision weights for all players
        self.weight_mat = np.random.default_rng().uniform(-1, 1, (self.num_agents, self.N, self.M + 1))
        
        # Initialize M random instances of weeks of attendance
        self.A_M = np.random.randint(0, self.num_agents-1, self.M)
        
#         # Initialize an array of indices for optimum strategies
#         self.S_star_ind = np.zeros(self.num_agents, dtype=int)
        
        # Initialize the current time step's attendance: 
        A_t = np.random.randint(0, self.num_agents-1)
        
    def make_decision(self, w_vec, A_vec, A_t):
        # Decision making for one person:
        # w_vec: [N x M+1], A_vec: [M x 1]
        assert w_vec.shape[1]-1 == A_vec.shape[0]
        S = np.zeros((self.N, 1))
        E = np.zeros((self.N, 1))
        for i in range(len(w_vec)):
            S[i] = 100 * (np.dot(w_vec[i,:-1], A_vec)) + w_vec[i,-1]
#             print(S[i].item().shape)
            E[i] = np.sum(np.abs(S[i].item() - np.append(A_t, A_vec)))
        
        # Select the strategy with the smallest error:
        s_ind = np.argmin(E)
        
        # Make Decision:
        if S[s_ind] <= self.THR:
            d = 1 # GO
        else:
            d = 0 # DON'T GO
            
        return S, E, d
        
    def step(self, n = 200):
        pass
        

In [72]:
# x = np.array((1,2,3,4))
# y = np.array((5,6))
1.63 - np.append(x,y)

array([ 0.63, -0.37, -1.37, -2.37, -3.37, -4.37])

In [73]:
env = El_Farol_env()

In [75]:
w_vec = np.squeeze(env.weight_mat[1, :, :])
A_vec = env.A_M
A_t = 44
# print(w_vec.shape)
# print(A_vec.shape)
# print(w_vec[1,:-1])
# s1 = (np.dot(w_vec[1,:-1], A_vec)) + w_vec[1,-1]
# print(s1)
# print(env.A_M.shape)
# print(A_vec)
ss, ee = env.make_decision(w_vec=w_vec, A_vec=A_vec, A_t=A_t)

In [79]:
ss

array([[ -6338.07950451],
       [-16032.79709578],
       [ -3384.80140973],
       [  -515.54721662],
       [ -7230.46056523],
       [ -6477.94171959],
       [ 17546.98919318],
       [  5374.9541724 ],
       [  2413.43798114],
       [ -4851.68268448]])

In [ ]:
env = El_Farol_env()

In [39]:
a_test = np.array((1,2))
w_test = a_test

S, E = env.make_decision(a_test,w_test, )

TypeError: make_decision() missing 1 required positional argument: 'A_t'

In [ ]:
env.weight_mat.shape

In [ ]:
np.random.randint(0,99,5)